## NumPy fundamentals

Here, I work through some parts of the numpy docs and make sense of it by trying code and asking LLMs for more detailed reasons for why things are as they are. Covered docs: <br>
     - [Indexing on ndarrays](https://numpy.org/doc/stable/user/basics.indexing.html) <br>

#### Shape, size and dimension
- dimensions (`ndim` attribute) is the number of axes (levels of indexing) in the array.
- the shape (`shape` attribute) is a tuple representing the size in each dimension
- the size (`size` attribute) is the number of elements in an array / part of the array


In [ ]:
import numpy as np
a = np.array([[1, 2, 3], [4, 5, 6]])
a
# array([[1, 2, 3],
#        [4, 5, 6]])

a.ndim # 2
a.shape # (2, 3)
a.size # 6

6

### Indexing on `ndarrays`
- the three kinds of indexing are basic indexing, advanced indexing and field access

#### Basic indexing

In [52]:
x = np.arange(10)
x # array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

x.shape = (2, 5)  # now x is 2-dimensional
x

array([[0, 1, 2, 3, 4],
       [5, 6, 7, 8, 9]])

In [ ]:
# these are the same:

x[0][-1] == x[0,-1]

# x[0,-1] is however more efficient, because the extraction happens in the same
# operation

np.True_

In [45]:
# indexing a multidimensional array with fewer indices than dimensions, we get a
# subdimensional array

# that's a view on the array x
x[0] # array([0, 1, 2, 3, 4])

x[0].shape


(5,)

In [ ]:
# slicing goes [start:stop:step]

x = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

x[1:7:2] # array([1, 3, 5])
x[-2:10] # array([8, 9])
x[-3:3:-1] # array([7, 6, 5, 4])
x[5:] # array([5, 6, 7, 8, 9])

array([5, 6, 7, 8, 9])

In [49]:
# if the number of objects in the selection tuple is less than the number of the
# dimensions, then `:` (or `np.newaxis`) is assumed for any subsequent dimensions:

x = np.array([[[1],[2],[3]], [[4],[5],[6]]])
x.shape # (2, 3, 1)
x[1:2] 
# here we are giving just one index (a slice on axis 0), and not saying anything about
# axes 1 and 2, which are filled up with `:` (like it was `x[1:2, :, :]`)

array([[[4],
        [5],
        [6]]])

In [ ]:
# equally, consider the ellipsis, that stands for several colons
x[1:2, :, :] == x[1:2, ...]

array([[[ True],
        [ True],
        [ True]]])

In [ ]:
# note on dimensions and axis:

x = np.array([[[1],[2],[3]], [[4],[5],[6]]])
x 
#  array([[[1],
#         [2],
#         [3]],
# 
#        [[4],
#         [5],
#         [6]]])

x.shape # (2, 3, 1)

# the axis are read from the outside to the inside:
# axis 0 contains 2 lists, thus `x.shape[0]` is 2
# axis 1 contains 3 lists, thus `x.shape[1]` is 3
# axis 3 contains 1 element, thus `x.shape[2]` is 1

# a more visual-friendly way to write this down is:
# [
#   [[1], [2], [3]],
#   [[4], [5], [6]]
# ]

# another way to think of this is that the outmost axis is the lastest to join the game:

# axis 0  (Depth / "stack"): the outermost list — 2 blocks → x[0], x[1]
# axis 1 (Rows): inside each block, 3 rows → x[0][0], x[0][1], x[0][2]
# axis 2 (Columns): each row has 1 column → x[0][0][0] is the number 1

(2, 3, 1)

In [ ]:
# newaxis and None can be used to add a new axis; the result is a view on x (note:
# np.newaxis is actually just an alias for None)

x[:, np.newaxis, :, :].shape

(2, 1, 3, 1)

In [ ]:
x.shape # x still stays the same then ...

(2, 3, 1)

In [ ]:
x = np.arange(5)

x # array([0, 1, 2, 3, 4])

x[:, np.newaxis] # adds new axis on the inside
# array([[0],
#        [1],
#        [2],
#        [3],
#        [4]])

x[np.newaxis, :]  # adds new axis from the outside
# array([[0, 1, 2, 3, 4]])

x[:, np.newaxis] + x[np.newaxis, :] # performs broadcasting (adding arrays of shape (5, 1) and (1, 5))

array([[0, 1, 2, 3, 4],
       [1, 2, 3, 4, 5],
       [2, 3, 4, 5, 6],
       [3, 4, 5, 6, 7],
       [4, 5, 6, 7, 8]])

#### Advanced indexing
- different indexing mechanism
- triggered when the selection object (the thing in the squared brackets) is a non-tuple
  sequence object, an ndarray (of data type integer or bool), or a tuple with at least
  one sequence object or ndarray (of data type integer or bool)
- always returns a copy

Warning from the docs: `x[(1, 2, 3),]` (advanced indexing) is fundamentally different than `x[(1, 2, 3)]` (basic indexing)! 🤯

In [69]:
# let's try this out:
x = np.arange(27).reshape(3,3,3)

x
# array([[[ 0,  1,  2],
#         [ 3,  4,  5],
#         [ 6,  7,  8]],

#        [[ 9, 10, 11],
#         [12, 13, 14],
#         [15, 16, 17]],

#        [[18, 19, 20],
#         [21, 22, 23],
#         [24, 25, 26]]])

# basic indexing
x[(2,2,2)]
# np.int64(26)

# advanced indexing
x[(2,2,2),]

array([[[18, 19, 20],
        [21, 22, 23],
        [24, 25, 26]],

       [[18, 19, 20],
        [21, 22, 23],
        [24, 25, 26]],

       [[18, 19, 20],
        [21, 22, 23],
        [24, 25, 26]]])

😱 :scream:

`x[(2,2,2),]` gets interpreted as repeated access to the element at index 2 at the
outmost axis

Attempt for an explanation: <br>
- `x[(2,2,2)]` basic indexing is a tuple which gets interpreted as one element per
  axis, like it was `x[2][2][2]`
- `x[(2,2,2),]` is a tuple with one element, and that element is another tuple, which
  gets interpreted as an array-like object for advanced indexing: it gets used as a list
  of indices along axis 0 (the outmost axis), like it was `x[2], x[2], x[2]` and
  returned as a single new array (or put differently, like it was `x[np.array([2,2,2])]`)

In [71]:
# let's try that out

x[2], x[2], x[2]

#here several arrays are returned, in advanced indexing it is all returnes as a single
#array

(array([[18, 19, 20],
        [21, 22, 23],
        [24, 25, 26]]),
 array([[18, 19, 20],
        [21, 22, 23],
        [24, 25, 26]]),
 array([[18, 19, 20],
        [21, 22, 23],
        [24, 25, 26]]))

In [72]:
# let's try that out

x[np.array([2,2,2])]

array([[[18, 19, 20],
        [21, 22, 23],
        [24, 25, 26]],

       [[18, 19, 20],
        [21, 22, 23],
        [24, 25, 26]],

       [[18, 19, 20],
        [21, 22, 23],
        [24, 25, 26]]])

In [ ]:
# again: that is not a view, it is a copy:

x[(2,2,2),].base

Other array-like elements that cause advanced indexing are:
- Python lists: `x[[0, 2]]`
- NumPy arrays: `x[np.array([0, 2])]`
- Boolean arrays: `x[np.array([True, False, True])]`
- Tuples of arrays: `x[[0,1],[1,0]]` (selects multiple points)

In [65]:
# example of advanced indexing using integers arrays (could be numpy arrays or lists
# containing integers that get interpreted as indexes)

x = np.arange(10, 1, -1)
x # array([10,  9,  8,  7,  6,  5,  4,  3,  2])

x[np.array([3, 3, 1, 8])] # array([7, 7, 9, 2])

x[np.array([3, 3, -3, 8])] # array([7, 7, 4, 2])

# the advanced indexing element here is that all these elements get selected from *the same* axis

array([7, 7, 4, 2])

In [76]:
# 2d example
x = np.array([[1, 2], [3, 4], [5, 6]])
x 
# array([[1, 2],
#        [3, 4],
#        [5, 6]])

x[np.array([1, -1])] 
# takes two elements (the element on position 1 and the element on position -1) from
# axis 0 (rows)

array([[3, 4],
       [5, 6]])

In [89]:
y = np.arange(35).reshape(5, 7)
y
# array([[ 0,  1,  2,  3,  4,  5,  6],
#        [ 7,  8,  9, 10, 11, 12, 13],
#        [14, 15, 16, 17, 18, 19, 20],
#        [21, 22, 23, 24, 25, 26, 27],
#        [28, 29, 30, 31, 32, 33, 34]])

y[np.array([0, 2, 4]), np.array([0, 1, 2])] 
# first index is for axis 0 (row), second for axis 1 (column)

array([ 0, 15, 30])

In [90]:
# if the index arrays don't have the same shape, they cannot be broadcast together and
# we get an error:

y[np.array([0, 2, 4]), np.array([0, 1])]

IndexError: shape mismatch: indexing arrays could not be broadcast together with shapes (3,) (2,) 

In [ ]:
# but a scalar could also be broadcast:

y[np.array([0, 2, 4]), 1] # here 1 is broadcast to all the columns

array([ 1, 15, 29])

In [93]:
# let's try the same with an additional dimension
y = np.arange(35).reshape(5, 7)
y = y[np.newaxis, :, :]
y
# array([[[ 0,  1,  2,  3,  4,  5,  6],
#         [ 7,  8,  9, 10, 11, 12, 13],
#         [14, 15, 16, 17, 18, 19, 20],
#         [21, 22, 23, 24, 25, 26, 27],
#         [28, 29, 30, 31, 32, 33, 34]]])

y[:, np.array([0, 2, 4]), np.array([0, 1, 2])] 
# first index is for axis 0 (batch), where we just select all, second index is for axis
# 1 (row), third for axis 2 (column)

array([[ 0, 15, 30]])

In [ ]:
# we can also only partially index an array with advanced indexing

y[:, np.array([0, 2, 4])] 
# only indexes rows (using all the batches), but leaves out the columns (meaning we use all of them)

array([[[ 0,  1,  2,  3,  4,  5,  6],
        [14, 15, 16, 17, 18, 19, 20],
        [28, 29, 30, 31, 32, 33, 34]]])

In [ ]:
# fancy indexing
# (selecting specific elements)

x = np.array([[1, 2], [3, 4], [5, 6]])
x[[0, 1, 2], [0, 1, 0]] 
# [0, 1, 2] means row 0, row 1, row 2 
# [0, 1, 0] means column 0, column 1, column 0

array([1, 4, 5])